# Import Packages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import datetime
from wordcloud import WordCloud

# Produce List Of Dates For Webscrape

In [5]:
DateList = []
temp = datetime.date(1958,8,4)
DateList.append(temp) 
while temp < datetime.date.today():
    temp = temp + datetime.timedelta(days=7)
    DateList.append(str(temp))
print(DateList[1])

1958-08-11


# Define Function To Pull Charts Data & Organise In To Data Frame

In [6]:
def Scrape(Date,Website):
    r=requests.get(str(Website))
    Data = BeautifulSoup(r.text,"html.parser")
    ArtistList = []
    for i in Data.find_all(class_="chart-list-item__artist"):
        ArtistList.append(re.search(".*(?=\n\s*</a>)|.*(?=[^</a>]</div>)",str(i))[0])
    TrackList = []
    for i in Data.find_all(class_="chart-list-item__title-text"):
        TrackList.append(re.search(".*(?=\n*</span>)",str(i))[0])
    Output = pd.DataFrame({"Track": TrackList,
                           "Artist": ArtistList,
                           "Rank": list(range(1, len(TrackList)+1)),
                           "Date": str(Date)})
    return(Output)

# Run Function For All Dates & Store In List

In [7]:
DataFrameList = []
for j, i in enumerate(DateList):
    if(j%100==0):
        print(str(j)+" Out Of ",str(len(DateList)))
    try:
        DataFrameList.append(Scrape(str(i),"https://www.billboard.com/charts/hot-100/"+str(i)+"/"))
    except:
        pass

0 Out Of  3177
100 Out Of  3177
200 Out Of  3177
300 Out Of  3177
400 Out Of  3177
600 Out Of  3177
700 Out Of  3177
800 Out Of  3177
900 Out Of  3177
1000 Out Of  3177
1100 Out Of  3177
1200 Out Of  3177
1300 Out Of  3177
1400 Out Of  3177
1500 Out Of  3177
1600 Out Of  3177
1700 Out Of  3177
1800 Out Of  3177
1900 Out Of  3177
2000 Out Of  3177
2100 Out Of  3177
2200 Out Of  3177
2300 Out Of  3177
2400 Out Of  3177
2500 Out Of  3177
2600 Out Of  3177
2700 Out Of  3177
2800 Out Of  3177
2900 Out Of  3177
3000 Out Of  3177
3100 Out Of  3177


# Join All Data Frames Together

In [8]:
Data = DataFrameList[0]
for j, i in enumerate(DataFrameList):
    Data = pd.concat([Data,i])
print(Data.shape)

(305885, 4)


# Create No Duplicate Song Data

In [9]:
DataNoDups = Data[["Track","Artist"]].drop_duplicates(keep="first")
DataNoDups = DataNoDups.reset_index(drop = True)
DataNoDups.shape

(38193, 2)

# Pull Lyrics For Unique Songs Where Possible

In [10]:
Lyrics = []
for i, j in enumerate(DataNoDups.Track):
    if(i%2000==0):
        print(str(i)+" Out Of ",str(len(DataNoDups.Track)))
    try:
        Website = "http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect?artist="+str(DataNoDups.Artist[i])+"&song="+str(DataNoDups.Track[i])
        r = requests.get(Website)
        Output = BeautifulSoup(r.text,"html.parser")
        Output = str(Output.find_all("lyric"))
        Lyrics.append(re.search("(?<=<lyric>).*(?=</lyric>)",Output,re.DOTALL)[0])
    except:
        Lyrics.append("Error")
    pass

0 Out Of  38193
2000 Out Of  38193
4000 Out Of  38193
6000 Out Of  38193
8000 Out Of  38193
10000 Out Of  38193
12000 Out Of  38193
14000 Out Of  38193
16000 Out Of  38193
18000 Out Of  38193
20000 Out Of  38193
22000 Out Of  38193
24000 Out Of  38193
26000 Out Of  38193
28000 Out Of  38193
30000 Out Of  38193
32000 Out Of  38193
34000 Out Of  38193
36000 Out Of  38193
38000 Out Of  38193


# Create Lyrics Lookup Table & Print Match Rate

In [11]:
print(len([Text for Text in Lyrics if len(Text) > 20]), " - Rows With Lyrics")
DataNoDups["Lyrics"] = Lyrics
DataNoDups["Temp"] = DataNoDups.Track + " " + DataNoDups.Artist

22714  - Rows With Lyrics


# Pull The Lyrics In To The Main Dataset And Print Match Rate

In [12]:
Data["Temp"] = Data.Track + " " + Data.Artist
Data = Data.merge(DataNoDups[["Temp","Lyrics"]], how = "left", on = "Temp")
Data = Data.drop(columns = "Temp")
print(len([Text for Text in Data.Lyrics if len(Text) > 20])/len(Data.Lyrics), " - % Rows With Lyrics")
Data.head()

0.6895467250764176  - % Rows With Lyrics


,Track,Artist,Rank,Date,Lyrics
0,Poor Little Fool,Ricky Nelson,1.0,1958-08-04,I used to play around with hearts\r\nthat hast...
1,Patricia,Perez Prado And His Orchestra,2.0,1958-08-04,"Instrumental, NO LYRICS"
2,Splish Splash,Bobby Darin,3.0,1958-08-04,splish splash\r\nI was taking a bath\r\nlong a...
3,Hard Headed Woman,Elvis Presley With The Jordanaires,4.0,1958-08-04,"Well a hard headed woman,\r\nA soft hearted ma..."
4,When,Kalin Twins,5.0,1958-08-04,Error


# Create No Duplicates Artist Data

In [47]:
ArtistData = list(Data["Artist"].drop_duplicates())
print(len(ArtistData))

11769


# Pull Genres For Artists In

In [14]:
Genres = []
for j, i in enumerate(ArtistData):
    if(j%1000==0):
        print(str(j)+" Out Of ",str(len(ArtistData)))
    try:
        if(re.search("With",i) !=None):
            temp = re.search(".*(?=With)",i)[0]
        elif(re.search("Feat",i) !=None):
            temp = re.search(".*(?=Feat)",i)[0]
        elif(re.search(",",i) !=None):
            temp = i.split(",")[0]
        elif(re.search("&",i) !=None):
            temp = re.search(".*(?=&)",i)[0]
        else:
            temp = i
        temp = temp.lstrip()
        Website = "https://www.allmusic.com/search/artists/" + str(temp)
        r = requests.get(Website,headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'})
        Output = BeautifulSoup(r.text,"html.parser")
        Output = str(Output.find_all(class_="genres")[0])
        Genres.append(re.search("(?<=>)\s*.*(?=</)",Output,re.MULTILINE)[0])
    except:
        Genres.append("Error")

0 Out Of  11769
1000 Out Of  11769
2000 Out Of  11769
3000 Out Of  11769
4000 Out Of  11769
5000 Out Of  11769
6000 Out Of  11769
7000 Out Of  11769
8000 Out Of  11769
9000 Out Of  11769
10000 Out Of  11769
11000 Out Of  11769


# Create Genre Lookup Table

In [15]:
ArtistData = pd.DataFrame({"Artists" : ArtistData})
ArtistData["Genres"] = Genres
print("Match Rate - ", 1 -len(ArtistData.loc[ArtistData.Genres == "Error"])/len(ArtistData.Artists))

Match Rate -  0.9688163820205625


# Pull The Genre In To The Main Dataset And Print Match Rate

In [16]:
Data = Data.drop_duplicates()
Data = Data.merge(ArtistData, how = "left", left_on="Artist", right_on = "Artists").drop(columns = "Artists")
print(len([Text for Text in Data.Genres if Text != "Error"])/len(Data.Genres), " - % Rows With Genres")

0.9847605343623788  - % Rows With Genres


# Clean Genres

In [17]:
GenresClean = []
for j, i in enumerate(Data.Genres):
    try:
        GenresClean.append(re.search("(?<=\n)\s*\w*/*\w*",str(i))[0])
    except:
        GenresClean.append("Error")
GenresClean = [Text.strip(' ') for Text in GenresClean]
GenresClean = np.where(np.array(GenresClean) == "R","R&B",GenresClean)
Data["GenresClean"] = GenresClean

# Clean Lyrics

In [18]:
LyricsClean = []
for i in Data.Lyrics:
    try:
        temp = i.replace("\r"," ")
        temp = temp.replace("\n"," ")
        temp = temp.replace("\\"," ")
        temp = temp.replace("   ", " ")
        temp = temp.replace("  ", " ")
        LyricsClean.append(temp)
    except:
        LyricsClean.append("Error")
Data["LyricsClean"] = LyricsClean

# Checkpoint Output Dataset

In [19]:
CleanData = Data.drop(columns = ["Lyrics","Genres"])
CleanData.to_csv("Data.csv")
print(str(round(len(CleanData.loc[CleanData.GenresClean != "Error"])/len(CleanData),4)) + " - Genre Match Rate")
print(str(round(len(CleanData.loc[CleanData.LyricsClean != "Error"])/len(CleanData),4)) +" - Lyrics Match Rate")
CleanData.head()

0.9848 - Genre Match Rate
0.9417 - Lyrics Match Rate


,Track,Artist,Rank,Date,GenresClean,LyricsClean
0,Poor Little Fool,Ricky Nelson,1.0,1958-08-04,Pop/Rock,I used to play around with hearts that hastene...
1,Patricia,Perez Prado And His Orchestra,2.0,1958-08-04,Latin,"Instrumental, NO LYRICS"
2,Splish Splash,Bobby Darin,3.0,1958-08-04,Pop/Rock,splish splash I was taking a bath long about a...
3,Hard Headed Woman,Elvis Presley With The Jordanaires,4.0,1958-08-04,Pop/Rock,"Well a hard headed woman, A soft hearted man B..."
4,When,Kalin Twins,5.0,1958-08-04,Pop/Rock,Error


# Read Data From Checkpoint

In [70]:
Data = pd.read_csv("Data.csv").drop(columns = "Unnamed: 0")
print(Data.shape)

(305785, 6)


# Clean Up Artists

In [71]:
CleanArtist = [str(i).strip().lower() for i in Data.Artist]

TidyArtist = []
j = 20000
for i in CleanArtist:
    CurrentArtist = str(i)
    if "and " in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("and ")]
    if "&" in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("&")]
    if "with " in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("with ")]    
    if "featuring " in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("featuring ")]
    if "his " in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("his ")]    
    if "wit " in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("wit ")]          
    if "x " in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("x ")]  
    if "+" in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("+")]  
    if "," in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find(",")]                      
    if "/" in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find("/")] 
    if ";" in i:
        CurrentArtist = CurrentArtist[:CurrentArtist.find(";")] 
    TidyArtist.append(CurrentArtist)

print(len(TidyArtist),len(Data.Artist))

Data["ArtistClean"] = TidyArtist
Data.head()

305785 305785


,Track,Artist,Rank,Date,GenresClean,LyricsClean,ArtistClean
0,Poor Little Fool,Ricky Nelson,1.0,1958-08-04,Pop/Rock,I used to play around with hearts that hastene...,ricky nelson
1,Patricia,Perez Prado And His Orchestra,2.0,1958-08-04,Latin,"Instrumental, NO LYRICS",perez prado
2,Splish Splash,Bobby Darin,3.0,1958-08-04,Pop/Rock,splish splash I was taking a bath long about a...,bobby darin
3,Hard Headed Woman,Elvis Presley With The Jordanaires,4.0,1958-08-04,Pop/Rock,"Well a hard headed woman, A soft hearted man B...",elvis presley
4,When,Kalin Twins,5.0,1958-08-04,Pop/Rock,Error,kalin twins


# Prepare Artist List

In [60]:
ArtistList = Data.ArtistClean.unique()
print(len(ArtistList))

7706


# Pull Genre From Wikipedia

In [61]:
GenreList = []
for j, i in enumerate(ArtistList):
    if(j%500 == 0):
        print(j)
    r = requests.get("https://en.wikipedia.org/w/index.php?search={}+Music+OR+Artist".format(str(i)), allow_redirects=True)
    Extract = BeautifulSoup(r.text, "html.parser")
    try:
        r = requests.get("https://en.wikipedia.org/"+str(re.search('(?<=href=\")\S*(?=\")',
                                                                    str(Extract.find_all(class_="mw-search-result-heading")[0]))[0]))
        Extract = BeautifulSoup(r.text, "html.parser")
        if Extract.find_all(class_="infobox vcard plainlist"):
            try:
                GenreList.append(re.findall('(?<=title\=").*?(?=\")',
                                                         re.search("Genres.*(?=</ul>)",
                                                                   " ".join(str(
                                                                       Extract.find_all(class_="infobox vcard plainlist")).split("\n")))[0])[0])
            except:
                try:
                    GenreList.append(re.search('(?<=title=).*?(?=\>)',
                            re.search("(?<=Genres)\D*",
                                                                       " ".join(str(
                                                                           Extract.find_all(class_="infobox vcard plainlist")).split("\n")))[0])[0])
                except:
                    GenreList.append(re.search("(?<=Genres).*?(?=</td>)",
                                                                       " ".join(str(
                                                                           Extract.find_all(class_="infobox vcard plainlist")).split("\n")))[0])
        elif Extract.find_all(class_="infobox biography vcard"):
            try:
                GenreList.append(re.findall('(?<=title\=").*?(?=\")',
                                                         re.search("Genres.*(?=</ul>)",
                                                                   " ".join(str(
                                                                   Extract.find_all(class_="infobox biography vcard")).split("\n")))[0])[0])
            except:
                GenreList.append("NO MATCH")
        elif Extract.find_all(class_="infobox vcard"):
            try:
                GenreList.append(re.findall('(?<=title\=").*?(?=\")',
                                                         re.search("Genres.*(?=</ul>)",
                                                                   " ".join(str(
                                                                   Extract.find_all(class_="infobox vcard")).split("\n")))[0])[0])
            except:
                GenreList.append("NO MATCH")
        else:
            r = requests.get("https://en.wikipedia.org/w/index.php?search={}+Music".format(str(i)), allow_redirects=True)
            Extract = BeautifulSoup(r.text, "html.parser")
            r = requests.get("https://en.wikipedia.org/"+str(re.search('(?<=href=\")\S*(?=\")',
                                                                    str(Extract.find_all(class_="mw-search-result-heading")[0]))[0]))
            Extract = BeautifulSoup(r.text, "html.parser")
            if Extract.find_all(class_="infobox vcard plainlist"):
                try:
                    GenreList.append(re.findall('(?<=title\=").*?(?=\")',
                                                             re.search("Genres.*(?=</ul>)",
                                                                       " ".join(str(
                                                                           Extract.find_all(class_="infobox vcard plainlist")).split("\n")))[0])[0])
                except:
                    try:
                        GenreList.append(re.search('(?<=title=).*?(?=\>)',
                                re.search("(?<=Genres)\D*",
                                                                           " ".join(str(
                                                                               Extract.find_all(class_="infobox vcard plainlist")).split("\n")))[0])[0])
                    except:
                        GenreList.append(re.search("(?<=Genres).*?(?=</td>)",
                                                                           " ".join(str(
                                                                               Extract.find_all(class_="infobox vcard plainlist")).split("\n")))[0])
            elif Extract.find_all(class_="infobox biography vcard"):
                try:
                    GenreList.append(re.findall('(?<=title\=").*?(?=\")',
                                                             re.search("Genres.*(?=</ul>)",
                                                                       " ".join(str(
                                                                       Extract.find_all(class_="infobox biography vcard")).split("\n")))[0])[0])
                except:
                    GenreList.append("NO MATCH")
            elif Extract.find_all(class_="infobox vcard"):
                try:
                    GenreList.append(re.findall('(?<=title\=").*?(?=\")',
                                                         re.search("Genres.*(?=</ul>)",
                                                                   " ".join(str(
                                                                   Extract.find_all(class_="infobox vcard")).split("\n")))[0])[0])
                except:
                    GenreList.append("NO MATCH")
            else:
                GenreList.append("NO MATCH")
    except:
        GenreList.append("NO MATCH")

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


# Build Artist Genre Lookup Table

In [62]:
print(len(GenreList), len(ArtistList))
ArtistList["Genre"] = GenreList
ArtistList.columns = ["Artist","Genre"]
ArtistList.head()

7706 7706


,Artist,Genre
0,ricky nelson,Rockabilly
1,perez prado,"""Mambo (music)"""
2,bobby darin,Jazz
3,elvis presley,Rock and roll
4,kalin twins,</th><td>Pop


# Clean Up New Genres

In [64]:
Rock = ['Rock music','"Rock music"','Alternative rock','Pop rock','"Rock and roll"','"Alternative rock"','Hard rock','Rock and roll','Indie rock','Progressive rock','Heavy metal music','"Hard rock"','"Soft rock"','Punk rock','Folk rock','"Folk rock"','Psychedelic rock','Southern rock','Post-grunge','"Psychedelic rock"','Alternative metal','"Rockabilly"','"Punk rock"','"Country rock"','Rockabilly','Art rock','Blues rock','Instrumental rock','"Indie Rock"','"Southern rock"','"Garage rock"','Indie folk','Country rock','Garage rock','"Heavy metal music"','Soft rock','Roots rock','Glam metal','Glam rock','Nu metal','Grunge','"Big band music"','"Big band"','"Glam metal"','"Hard bop"','"Progressive rock"','Emo']
Religous = ['"Gospel music"','Gospel music','Contemporary Christian music','"Contemporary Christian music"','"Contemporary worship music"']
Reggae = ['"Reggae"','Reggae','Reggaeton','Dancehall','Reggae fusion']
RnB = ['Contemporary R&amp;B','Rhythm and blues','Funk','R&amp;B','"Funk"','</th><td><a href="/wiki/Contemporary_R%26B" title="Contemporary R&amp;B">R&amp;B</a>','Doo-wop','"Doo Wop"','Alternative R&amp;B','"Doo-wop"','"Beat music"','</th><td><a href="/wiki/Contemporary_R%26B" title="Contemporary R&amp;B">R&amp;B</a>, <a href="/wiki/Soul_music" title="Soul music">soul</a>','Rhythm &amp; blues','</th><td><a class="mw-redirect" href="/wiki/R%26B" title="R&amp;B">R&amp;B</a>','</th><td><a href="/wiki/Contemporary_R%26B" title="Contemporary R&amp;B">R&amp;B</a>, <a href="/wiki/Hip_hop_music" title="Hip hop music">Hip hop</a>','</th><td><a href="/wiki/Contemporary_R%26B" title="Contemporary R&amp;B">R&amp;B</a>, <a href="/wiki/Soul_music" title="Soul music">soul</a>, <a href="/wiki/Neo_soul" title="Neo soul">neo soul</a>','</th><td><a href="/wiki/Contemporary_R%26B" title="Contemporary R&amp;B">R&amp;B</a>, <a href="/wiki/New_jack_swing" title="New jack swing">new jack swing</a>']
Pop = ['Pop music','"Pop music"','"Pop rock"','New wave music','K-pop','"New wave music"','Traditional pop music','"Disco"','Synth-pop','Pop punk','Latin pop','Indie pop','Dance-pop','"Traditional pop music"','"Dance-pop"','</th><td>Pop','Disco','"Synth-pop"','Art pop','Film score','"Film score"','"Popular music"','Synthpop','Country pop','"Traditional pop"','"Country pop"','Adult contemporary music','Power pop','Bubblegum pop','Dance music','"Bubblegum pop"','"Sunshine pop"','"Ska"','Traditional pop']
HipHop = ['Hip hop music','"Hip hop music"','"Freestyle music"','West Coast hip hop','Alternative hip hop','Hip hop','Hip hop soul','"Southern hip hop"','"West Coast hip hop"','Pop-rap','Trip hop','"Alternative hip hop"','East Coast hip hop','Korean hip hop','Southern hip hop']
Folk = ['Observational comedy','Folk music','"Folk music"','"Latin pop"','Latin trap','Bachata (music)','Americana (music)','Broadway theatre','Contemporary folk music','"Americana (music)"']
Electro = ['Electronic dance music','House music','Electropop','"Eurodance"','"House music"','Electronic music','"Electronica"','Electronica','"Dance music"','Eurodance','"Electronic music"','"Synthpop"']
Country = ['Easy listening','"Easy listening"','Country music','"Country music"','Neotraditional country','Neotraditionalist country']
Blues = ['"Rhythm and blues"','"Jazz"','Soul music','"Soul music"','Jazz','Blues','"Blues"','"Blues rock"','Blue-eyed soul','"Blue-eyed soul"','Neo soul','"Chicago soul"','"Southern soul"']

In [65]:
GenreClean = []

for i in ArtistList.Genre:
    if i in Rock:
        GenreClean.append("Rock")
    elif i in Religous:
        GenreClean.append("Religous")
    elif i in Reggae:
        GenreClean.append("Reggae")
    elif i in RnB:
        GenreClean.append("R&B")
    elif i in Pop:
        GenreClean.append("Pop")
    elif i in HipHop:
        GenreClean.append("HipHop")
    elif i in Folk:
        GenreClean.append("Folk")
    elif i in Electro:
        GenreClean.append("Electro")
    elif i in Country:
        GenreClean.append("Country")
    elif i in Blues:
        GenreClean.append("Blues")
    else:
        GenreClean.append("Error")

ArtistList["GenreClean"] = GenreClean
ArtistList = ArtistList.drop(columns = "Genre")
print(ArtistList.GenreClean.value_counts())
ArtistList.head(3)

Error       2541
Rock        1185
Pop         1064
HipHop       856
R&B          677
Country      547
Blues        454
Electro      160
Folk         109
Reggae        76
Religous      37
Name: GenreClean, dtype: int64


,Artist,GenreClean
0,ricky nelson,Rock
1,perez prado,Error
2,bobby darin,Blues


# Pull In Second Genre

In [77]:
Data = Data.merge(ArtistList, how = "left", left_on = "ArtistClean", right_on = "Artist")
pd.crosstab(Data.GenreClean, Data.GenresClean)

GenresClean,Avant,Blues,Children,Classical,Comedy/Spoken,Country,Easy,Electronic,Error,Folk,...,Jazz,Latin,New,Pop/Rock,R&B,Rap,Reggae,Religious,Stage,Vocal
GenreClean,,,,,,,,,,,,,,,,,,,,,
Blues,0,694,0,13,0,0,335,51,183,0,...,933,185,0,2509,12790,152,126,0,0,550
Country,0,19,15,0,32,27280,129,552,158,111,...,52,20,0,2544,265,172,67,20,0,309
Electro,0,14,0,1,0,0,0,2742,81,14,...,11,0,0,1437,338,122,0,0,0,0
Error,0,248,38,160,471,2232,976,2416,1418,181,...,925,717,80,24671,11991,5485,237,269,505,2074
Folk,0,0,0,13,20,40,0,42,42,704,...,0,116,0,1921,130,11,0,16,35,505
HipHop,26,63,1,0,0,254,80,407,400,0,...,37,0,0,2377,2910,25108,185,25,4,0
Pop,0,0,100,21,34,810,215,865,499,11,...,244,1237,7,37920,6435,1812,74,99,62,4038
R&B,0,221,6,3,32,209,39,663,346,393,...,155,416,0,7519,31178,446,7,56,0,350
Reggae,0,0,0,0,0,15,0,23,89,0,...,4,245,0,546,248,176,977,0,0,0


# View Distribution And Errors Of Genres

In [80]:
Data.GenreClean.value_counts()

Rock        59103
Error       55272
Pop         54563
R&B         42058
HipHop      31877
Country     31828
Blues       18529
Electro      4767
Folk         3611
Reggae       2326
Religous      905
Name: GenreClean, dtype: int64

In [81]:
Data.GenresClean.value_counts()

Pop/Rock         134458
R&B               68743
Rap               33931
Country           32256
Vocal              8485
Electronic         8200
Error              4660
Latin              2981
Jazz               2537
Easy               1841
Reggae             1700
Folk               1530
Blues              1284
Religious          1046
Stage               646
Comedy/Spoken       594
International       391
Classical           216
Children            160
New                  87
Avant                26
Holiday              13
Name: GenresClean, dtype: int64

# Tidy Instrumental Lyrics

In [88]:
LyricsClean = []
for i in Data.LyricsClean:
    if (len(str(i)) < 30) & (len(str(i))>5):
        LyricsClean.append("Instrumental")
    elif len(str(i)) <= 5:
        LyricsClean.append("Error")
    else:
        LyricsClean.append(i)

Data["Lyrics"] = LyricsClean

# Tidy Data Frame

In [97]:
Data = Data.drop(columns = "Artist_y")
Data.columns = ["Track","Artist1","Rank","Date","Genre1","LyricsOriginal","Artist2","Genre2","LyricsClean"]
Data = Data[["Rank","Date","Track","Artist1","Artist2","Genre1","Genre2","LyricsOriginal","LyricsClean"]]
Data = Data.replace("Error",np.nan)
Data.Artist2 = [str(i).title() for i in Data.Artist2]
Data.head()

,Rank,Date,Track,Artist1,Artist2,Genre1,Genre2,LyricsOriginal,LyricsClean
0,1.0,1958-08-04,Poor Little Fool,Ricky Nelson,Ricky Nelson,Pop/Rock,Rock,I used to play around with hearts that hastene...,I used to play around with hearts that hastene...
1,2.0,1958-08-04,Patricia,Perez Prado And His Orchestra,Perez Prado,Latin,NaN,"Instrumental, NO LYRICS",Instrumental
2,3.0,1958-08-04,Splish Splash,Bobby Darin,Bobby Darin,Pop/Rock,Blues,splish splash I was taking a bath long about a...,splish splash I was taking a bath long about a...
3,4.0,1958-08-04,Hard Headed Woman,Elvis Presley With The Jordanaires,Elvis Presley,Pop/Rock,Rock,"Well a hard headed woman, A soft hearted man B...","Well a hard headed woman, A soft hearted man B..."
4,5.0,1958-08-04,When,Kalin Twins,Kalin Twins,Pop/Rock,Pop,NaN,NaN


In [98]:
print(Data.shape[0], Data.Genre1.count(), Data.Genre2.count(), Data.LyricsClean.count(), Data.dropna().shape[0])
print(Data.Artist2[Data.Rank <= 3].value_counts())

305785 301125 249567 213094 176042


# Output Fully Cleaned Dataset For Modelling

In [103]:
Data.to_csv("CleanData.csv")